In [1]:
import numpy as np
import pandas as pd
from scipy import stats

### Задача 1.

#### Используя данные о хоккеистах, проверьте, является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие:
#### 1) роста
#### 2) BMI

In [2]:
df = pd.read_csv('hockey_players.csv', encoding='ISO-8859-1')

df.head()

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
0,2001,RUS,10,tverdovsky oleg,D,L,185,84.0,1976-05-18,anaheim mighty ducks,24.952772,1976,24.543462
1,2001,RUS,2,vichnevsky vitali,D,L,188,86.0,1980-03-18,anaheim mighty ducks,21.119781,1980,24.332277
2,2001,RUS,26,petrochinin evgeni,D,L,182,95.0,1976-02-07,severstal cherepovetal,25.229295,1976,28.680111
3,2001,RUS,28,zhdan alexander,D,R,178,85.0,1971-08-28,ak bars kazan,29.675565,1971,26.827421
4,2001,RUS,32,orekhovsky oleg,D,R,175,88.0,1977-11-03,dynamo moscow,23.490760,1977,28.734694


In [3]:
# в этой задаче для проверки значимого отличия подойдёт однофакторный дисперсионный анализ
# создадим функцию для подсчёта значения статистики однофакторного дисперсионного анализа:

def anova(*y_):
    y = np.concatenate([*y_])
    y_mean = y.mean()

    SS_b = sum((y_i.mean() - y_mean) ** 2 * y_i.size for y_i in y_)
    SS_w = sum(((y_i - y_i.mean()) ** 2).sum() for y_i in y_)

    k = len(y_)
    n = y.size

    k1 = k - 1
    k2 = n - k

    sigma2_b = SS_b / k1
    sigma2_w = SS_w / k2

    F = sigma2_b / sigma2_w

    return F

In [4]:
# посмотрим все уникальные значения стран в датасете:

df['country'].unique()

array(['RUS', 'AUT', 'BLR', 'CAN', 'CZE', 'FIN', 'GER', 'ITA', 'JPN',
       'LAT', 'NOR', 'SUI', 'SVK', 'SWE', 'UKR', 'USA', 'POL', 'SLO',
       'DEN', 'FRA', 'KAZ', 'HUN'], dtype=object)

In [5]:
countries = ['FIN', 'NOR', 'DEN'] # отбираем список необходимых стран

# проверяем их наличие:

for c in countries:
    if c in df['country'].values:
        print(f'{c} есть в списке стран')
    else:
        print(f'{c} отсутствует в списке стран')

FIN есть в списке стран
NOR есть в списке стран
DEN есть в списке стран


### 1) Рост

In [6]:
# отбираем все Y роста по странам и считаем значение статистики:

y_height = [df.loc[df['country'] == country, 'height'] for country in countries]

F = anova(*y_height)

F

4.273207343917213

In [7]:
# найдём критическую область:

k = len(countries)
n = sum(y.size for y in y_height)

k1 = k - 1
k2 = n - k

alpha = 0.05

t = stats.f.ppf(1 - alpha, k1, k2)

t

3.0045220661840073

In [8]:
# проверим P-значение при помощи функции f_oneway из модуля stats (и сравним значение статистики):

stats.f_oneway(*y_height)

F_onewayResult(statistic=4.273207343917114, pvalue=0.014186614738624594)

In [9]:
# значение статистики попало в критическую область, а P-значение меньше, чем alpha
# таким образом, гипотезу H0 можно отвергнуть, принимаем H1
# соответственно, у роста хоккеистов из Финляндии, Норвегии и Дании есть значимое отличие

### 2) BMI

In [10]:
# отбираем все Y BMI по странам и считаем значение статистики:

y_bmi = [df.loc[df['country'] == country, 'bmi'] for country in countries]

F = anova(*y_bmi)

F

36.05816727743814

In [11]:
# найдём критическую область:

k = len(countries)
n = sum(y.size for y in y_bmi)

k1 = k - 1
k2 = n - k

alpha = 0.05

t = stats.f.ppf(1 - alpha, k1, k2)

t

3.0045220661840073

In [12]:
# проверим P-значение при помощи функции f_oneway из модуля stats (и сравним значение статистики):

stats.f_oneway(*y_bmi)

F_onewayResult(statistic=36.058167277438685, pvalue=7.369962129725083e-16)

In [13]:
# значение статистики попало в критическую область, а P-значение меньше, чем alpha
# таким образом, гипотезу H0 можно отвергнуть, принимаем H1
# соответственно, у BMI хоккеистов из Финляндии, Норвегии и Дании есть значимое отличие

### Ответ: как у роста, так и у BMI среди хоккеистов из Финляндии, Норвегии и Дании есть значимое отличие.